In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.2'
spark_version = 'spark-3.5.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [5]:
# 2. Create a temporary view of the DataFrame.

# Add file to Spark context
spark.sparkContext.addFile(url)

# reading csv file
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

#creating temp view
df.createOrReplaceTempView("home_sales")



In [8]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

query1 = spark.sql("""
    SELECT YEAR(date) AS Year, ROUND(AVG(price), 2) AS Average_Price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY YEAR(date)
    ORDER BY Year
""")

# 3. Show the results
result.show()


+----------+-------------+
|Year_Built|Average_Price|
+----------+-------------+
|      2019|    287287.82|
|      2020|    294204.16|
|      2021|    294211.46|
|      2022|    292725.69|
+----------+-------------+



In [10]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

query2 = spark.sql("""
    SELECT YEAR(date_built) AS Year_Built, ROUND(AVG(price), 2) AS Average_Price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY Year_Built
    ORDER BY Year_Built
""")

# 3. Show the results
query2.show()


+----------+-------------+
|Year_Built|Average_Price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [11]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

query3 = spark.sql("""
    SELECT YEAR(date_built) AS Year_Built, ROUND(AVG(price), 2) AS Average_Price
    FROM home_sales
    WHERE bedrooms = 3
      AND bathrooms = 3
      AND floors = 2
      AND sqft_living >= 2000
    GROUP BY Year_Built
    ORDER BY Year_Built
""")

# Show the results
query3.show()


+----------+-------------+
|Year_Built|Average_Price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [12]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

# SQL query to calculate the average price per view rating
query4 = spark.sql("""
    SELECT view AS View_Rating, ROUND(AVG(price), 2) AS Average_Price
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY View_Rating DESC
""")

# Show the results
query4.show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----------+-------------+
|View_Rating|Average_Price|
+-----------+-------------+
|         99|   1061201.42|
|         98|   1053739.33|
|         97|   1129040.15|
|         96|   1017815.92|
|         95|    1054325.6|
|         94|    1033536.2|
|         93|   1026006.06|
|         92|    970402.55|
|         91|   1137372.73|
|         90|   1062654.16|
|         89|   1107839.15|
|         88|   1031719.35|
|         87|    1072285.2|
|         86|   1070444.25|
|         85|   1056336.74|
|         84|   1117233.13|
|         83|   1033965.93|
|         82|    1063498.0|
|         81|   1053472.79|
|         80|    991767.38|
+-----------+-------------+
only showing top 20 rows

--- 1.2788326740264893 seconds ---


In [13]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [14]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [15]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

cached_query = spark.sql("""
    SELECT view AS View_Rating, ROUND(AVG(price), 2) AS Average_Price
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY View_Rating DESC
""")

# Show the results
cached_query.show()



print("--- %s seconds ---" % (time.time() - start_time))


+-----------+-------------+
|View_Rating|Average_Price|
+-----------+-------------+
|         99|   1061201.42|
|         98|   1053739.33|
|         97|   1129040.15|
|         96|   1017815.92|
|         95|    1054325.6|
|         94|    1033536.2|
|         93|   1026006.06|
|         92|    970402.55|
|         91|   1137372.73|
|         90|   1062654.16|
|         89|   1107839.15|
|         88|   1031719.35|
|         87|    1072285.2|
|         86|   1070444.25|
|         85|   1056336.74|
|         84|   1117233.13|
|         83|   1033965.93|
|         82|    1063498.0|
|         81|   1053472.79|
|         80|    991767.38|
+-----------+-------------+
only showing top 20 rows

--- 0.7241876125335693 seconds ---


In [16]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").parquet("/path/to/output/home_sales_partitioned")

In [17]:
# 11. Read the parquet formatted data.
partitioned_df = spark.read.parquet("/path/to/output/home_sales_partitioned")
partitioned_df.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [18]:
# 12. Create a temporary table for the parquet data.
partitioned_df.createOrReplaceTempView("home_sales_parquet")


In [19]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

parquet_query = spark.sql("""
    SELECT view AS View_Rating, ROUND(AVG(price), 2) AS Average_Price
    FROM home_sales_parquet
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY View_Rating DESC
""")

# Show the results
parquet_query.show()


print("--- %s seconds ---" % (time.time() - start_time))

+-----------+-------------+
|View_Rating|Average_Price|
+-----------+-------------+
|         99|   1061201.42|
|         98|   1053739.33|
|         97|   1129040.15|
|         96|   1017815.92|
|         95|    1054325.6|
|         94|    1033536.2|
|         93|   1026006.06|
|         92|    970402.55|
|         91|   1137372.73|
|         90|   1062654.16|
|         89|   1107839.15|
|         88|   1031719.35|
|         87|    1072285.2|
|         86|   1070444.25|
|         85|   1056336.74|
|         84|   1117233.13|
|         83|   1033965.93|
|         82|    1063498.0|
|         81|   1053472.79|
|         80|    991767.38|
+-----------+-------------+
only showing top 20 rows

--- 1.0424587726593018 seconds ---


In [20]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [21]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")

# Print the result
print(f"Is the 'home_sales' table cached? {is_cached}")


Is the 'home_sales' table cached? False
